In [5]:
import json

# Demonstration of how to load a file that contains secrets without accidentally leaking those secrets
with open('credentials.json') as f:
    data = json.load(f)

    secret_key = data['mongodb']

# We can safely print the length of the secret key. That won't leak any sensitive information.
print(f"My secret key is {len(secret_key)} characters in length.")

My secret key is 68 characters in length.


In [4]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 697.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 2.3 MB/s eta 0:00:0000:0100:01
DEPRECATION: pytorch-lightning 1.8.3.post0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [12]:
import pymongo
import certifi

# Connect to the database using known good certificates
client = pymongo.MongoClient(secret_key, tlsCAFile=certifi.where())
print(f"Using MongoDB version {client.server_info()['version']}.")

# Check what databases exist on this server
all_databases = client.list_database_names()
print(f"This MongoDB server has the databases {all_databases}")

# If we know the correct database to talk to, we connect like this:
data320 = client['data320']

# Here is the list of collections within my database
all_collections = data320.list_collection_names()
print(f"This database has the collections {all_collections}")

Using MongoDB version 7.0.8.
This MongoDB server has the databases ['data320', 'sample_mflix', 'admin', 'local']
This database has the collections ['imdb', 'movies']


In [15]:
import pandas as pd
import re

# Retrieve all records from a collection - this can be a large amount of data!
cursor = data320["imdb"].find({"release_date": re.compile("2010")})

# Convert this information into a Pandas dataframe
imdb = pd.DataFrame(cursor)

# Make sure we've read the information correctly
imdb.head()


,_id,id,title,runtime,user_rating,votes,mpaa_rating,release_date,budget,opening_weekend,gross_sales,genres,cast,director,producer,company
0,662870e0c5e83e4c458c5c1a,1196141,Diary of a Wimpy Kid,94,6.2,49261,TV-PG::(V),3/18/2010,15000000.0,22126166.0,7.570050e+07,"Comedy, Drama, Family","Zachary Gordon, Robert Capron, Rachael Harris,...",Thor Freudenthal,"Nina Jacobson, Jeff Kinney, Brad Simpson, Etha...","Color Force, Dayday Films, Dune Entertainment III"
1,662870e0c5e83e4c458c5c1b,1375666,Inception,148,8.8,2282266,"TV-14::(DLV, TV Rating.)",7/8/2010,160000000.0,62785337.0,8.255328e+08,"Action, Adventure, Sci-Fi, Thriller","Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",Christopher Nolan,"Zakaria Alaoui, John Bernard, Chris Brigham, J...","Warner Bros., Legendary Entertainment, Syncopy"
2,662870e0c5e83e4c458c5c20,435761,Toy Story 3,103,8.3,818143,G,6/12/2010,190000000.0,110307189.0,1.068880e+09,"Animation, Adventure, Comedy, Family, Fantasy","Tom Hanks, Tim Allen, Joan Cusack, Ned Beatty,...",Lee Unkrich,"Darla K. Anderson, John Lasseter, Nicole Parad...","Walt Disney Pictures, Pixar Animation Studios"
3,662870e0c5e83e4c458c5c21,926084,Harry Potter and the Deathly Hallows: Part 1,146,7.7,534619,PG-13,11/11/2010,150000000.0,NaN,9.602833e+08,"Adventure, Family, Fantasy, Mystery","Bill Nighy, Emma Watson, Richard Griffiths, Ha...",David Yates,"David Barron, David Heyman, Tim Lewis, J.K. Ro...","Warner Bros., Heyday Films"
4,662870e0c5e83e4c458c5c22,1285016,The Social Network,120,7.8,686673,TV-14,9/24/2010,40000000.0,22445653.0,2.249203e+08,"Biography, Drama","Jesse Eisenberg, Rooney Mara, Bryan Barter, Du...",David Fincher,"Dana Brunetti, Ceán Chaffin, Jim Davidson, Mic...","Columbia Pictures, Relativity Media, Scott Rud..."


In [16]:
# convert release dates to datetime type
imdb.release_date = pd.to_datetime(imdb.release_date, errors='coerce')
# convert budget and runtimer to numeric type
imdb.budget = pd.to_numeric(imdb.budget, errors='coerce')
imdb.runtime = pd.to_numeric(imdb.runtime, errors='coerce')

In [1]:
imdb.head()

NameError: name 'imdb' is not defined